In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd  
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score

In [ ]:

data=pd.read_csv('../input/car-price-prediction/CarPrice_Assignment.csv')
data.head()

In [ ]:
data.info()

In [ ]:
data=data.drop(['car_ID','symboling','CarName'],axis=1)
data.info()

In [ ]:
import seaborn as sns
sns.heatmap(data.corr())

In [ ]:
data1=pd.get_dummies(data,drop_first=True)
X=(data1.drop('price',axis=1)).iloc[:,:].values
y=data.iloc[:,-1].values

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

regressor=LinearRegression()
regressor.fit(X_train,y_train)

y_pred=regressor.predict(X_test)
print(r2_score(y_test,y_pred))

In [ ]:
#Building the optimal model with BackwardElimination 

def BackwardElimination(X,y):
    import statsmodels.api as stm
    import numpy as np
    
    X=np.append(arr=np.ones((X.shape[0],1)).astype(int),values=X,axis=1)
    ColumnIndexList=[i for i in range(X.shape[1])]
    X_opt=X[:,ColumnIndexList]
    #Step2 : Fit the model with all possible predictors
    regressor_opt=stm.OLS(y,X_opt).fit()
    #Step3:Consider the prdictor with the highest P value if P>sl=0.05 Go to step 4 else Finish 
    pValuesList=list(regressor_opt.pvalues)
   
    while(True):
        verification=False
        for e in pValuesList:
            if e>0.04 :   
                ColumnIndexList.pop(pValuesList.index(e))
                pValuesList.remove(e)
                X_opt=X[:,ColumnIndexList]
                regressor_opt=stm.OLS(y,X_opt).fit()
            else :
                continue
        pValuesList=list(regressor_opt.pvalues)
        for e in pValuesList:
            if e>0.05 :
                verification=True
        if verification==False :
            break 
        else :
            continue
    return(pValuesList,regressor_opt,X_opt)

In [ ]:
pValuesList,regressor_OLS,X_opt=BackwardElimination(X,y)
regressor_OLS.summary()
y_OLS=regressor_OLS.predict(X_opt)

In [ ]:
print(r2_score(y_OLS,y))